In [80]:
import numpy as np
from quaos.core.paulis import PauliSum, PauliString
from quaos.core.circuits.target import find_map_to_target_pauli_sum
from quaos.core.circuits.gates import Gate
from quaos.utils import get_linear_dependencies
from quaos.graph_utils import plot_group_graph, find_swappable_pairs

First we make a random Pauli Sum, and obtain a list of the linearly independent rows

In [102]:
# parameters
n_paulis = 45
n_qudits = 5
d = 2
n_weights = 2 # number of different weights

dimensions = [d] * n_qudits
# make a random pauli sum
H = PauliSum.from_random(n_paulis, n_qudits, dimensions, rand_weights=False)

# this bit makes sections of H have different weights
weights = np.empty(n_paulis, dtype=int)
section_size = n_paulis // n_weights
for i in range(n_weights):
    start = i * section_size
    end = (i + 1) * section_size if i < n_weights - 1 else n_paulis
    weights[start:end] = i + 1

H.weights = weights
# print(H.symplectic())

# obtain linearly independent rows - note this fails sometimes as there is a solver in it that
# is not that robust. The Clifford approach is more robust as it creates a simple basis where the
# dependencies can be easily read.
independent_paulis, dependencies = get_linear_dependencies(H.symplectic(), d)

print(independent_paulis)
print(dependencies)

[0, 1, 2, 3, 4, 5, 6, 7, 9, 10]
{8: [(1, 1), (3, 1), (4, 1), (6, 1)], 11: [(0, 1), (3, 1), (5, 1), (6, 1), (7, 1)], 12: [(1, 1), (2, 1), (6, 1)], 13: [(1, 1), (2, 1), (3, 1), (4, 1), (7, 1), (9, 1)], 14: [(0, 1), (4, 1), (9, 1), (10, 1)], 15: [(0, 1), (1, 1), (3, 1)], 16: [(0, 1), (1, 1), (4, 1), (5, 1), (6, 1)], 17: [(1, 1), (3, 1), (5, 1), (9, 1)], 18: [(0, 1), (3, 1), (6, 1), (7, 1)], 19: [(3, 1), (5, 1), (7, 1)], 20: [(0, 1), (1, 1), (2, 1), (4, 1), (5, 1), (6, 1), (10, 1)], 21: [(0, 1), (1, 1), (4, 1), (7, 1), (9, 1), (10, 1)], 22: [(1, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (9, 1)], 23: [(0, 1), (1, 1), (4, 1), (10, 1)], 24: [(0, 1), (1, 1), (3, 1), (4, 1), (6, 1), (9, 1), (10, 1)], 25: [(0, 1), (4, 1), (5, 1), (6, 1), (10, 1)], 26: [(0, 1), (6, 1)], 27: [(2, 1), (9, 1), (10, 1)], 28: [(0, 1), (1, 1), (2, 1), (4, 1)], 29: [(0, 1), (1, 1), (3, 1), (6, 1), (10, 1)], 30: [(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (9, 1)], 31: [(0, 1), (1, 1), (2, 1), (3, 1), (7, 1), (9, 1

We now solve for possible permutations which leave the dependencies unchanged

To do so we make a dictionary with labels of the coefficients

***Note this will only work for qubits for now, as for qudits we also need to account for the fact that a dependency can be made of $P_i + kP_j$ for some integer $k < d$***

In [103]:
cs = H.weights

graph_dict = {}

for i in independent_paulis:
    key = cs[i]
    if key in graph_dict:
        graph_dict[key].append([i])
    else:
        graph_dict[key] = [[i]]

for i in dependencies.keys():
    key = cs[i]
    dependency = dependencies[i]
    dependence_indices = [x[0] for x in dependency]
    dependence_multiplicities = [x[1] for x in dependency]  # this will be needed for qudits! always 1 for now
    if key in graph_dict:
        graph_dict[key].append(dependence_indices)
    else:
        graph_dict[key] = [dependence_indices]

print(graph_dict)



{np.int64(1): [[0], [1], [2], [3], [4], [5], [6], [7], [9], [10], [1, 3, 4, 6], [0, 3, 5, 6, 7], [1, 2, 6], [1, 2, 3, 4, 7, 9], [0, 4, 9, 10], [0, 1, 3], [0, 1, 4, 5, 6], [1, 3, 5, 9], [0, 3, 6, 7], [3, 5, 7], [0, 1, 2, 4, 5, 6, 10], [0, 1, 4, 7, 9, 10]], np.int64(2): [[1, 3, 4, 5, 6, 7, 9], [0, 1, 4, 10], [0, 1, 3, 4, 6, 9, 10], [0, 4, 5, 6, 10], [0, 6], [2, 9, 10], [0, 1, 2, 4], [0, 1, 3, 6, 10], [1, 2, 3, 4, 5, 6, 9], [0, 1, 2, 3, 7, 9], [2, 4, 6], [2, 3, 4, 9], [0, 1, 4, 6, 7], [0, 5, 6, 9], [0, 2, 3, 5, 6, 7, 9, 10], [1, 3, 5, 6, 9], [0, 1, 2, 3, 4, 5, 6, 9], [0, 1, 2, 3, 6, 7, 9], [0, 2, 3, 7, 9], [0, 1, 2, 4, 5, 10], [0, 2, 3, 4, 6, 7], [2, 3, 4, 7, 9, 10], [0, 3, 5, 9]]}


We then test to see if any indexes have an automorphism that can be found via a simple swap

***There is an additional condition - the pairs when swapped must leave the symplectic product matrix invariant...***
As there is always going to be a reasonably small number of pairs, we can probably check the combinatorial number of
options for this...

In [104]:
pairs = find_swappable_pairs(graph_dict)
print(pairs)

[]


Now we choose a target - for this example, performing all of these swaps

We then find the symplectic which maps the original H to H_target

In [101]:
# make target
H_target = H.copy()
found = False
for p in pairs:  # This is inefficient - could be done better by checking only the commutation of Paulis to be swapped
    H_target = H.copy()
    H_target.swap_paulis(p[0], p[1])
    if np.array_equal(H_target.symplectic_product_matrix(), H.symplectic_product_matrix()):
        found = True
        print("Found automorphism")
        break

if not found:
    print("No automorphism found - there are only non-Pauli symmetries if the rank of H is < 2n")
    H_target = H.copy()


# find F
# It may be here that we need to use only the paulis that are linearly independent
H_indep = H[independent_paulis]
H_t_indep = H_target[independent_paulis]

F, _, _, _ = find_map_to_target_pauli_sum(H_indep, H_t_indep)

# print(F)
# check F

print(H_target.symplectic())
print(H.symplectic() @ F % d)
print(np.array_equal(H_target.symplectic(), H.symplectic() @ F % d))
print(F)


No automorphism found - there are only non-Pauli symmetries if the rank of H is < 2n
[[1 0 1 1 1 0]
 [1 1 0 1 0 1]
 [0 1 1 1 0 0]
 [0 0 1 0 1 0]
 [1 0 1 1 1 1]
 [1 0 0 1 0 1]
 [1 1 0 0 0 0]]
[[1 0 1 1 1 0]
 [1 1 0 1 0 1]
 [0 1 1 1 0 0]
 [0 0 1 0 1 0]
 [1 0 1 1 1 1]
 [1 0 0 1 0 1]
 [1 1 0 0 0 0]]
True
[[1 0 0 0 0 0]
 [0 1 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 1 0 0]
 [0 0 0 0 1 0]
 [0 0 0 0 0 1]]
